In [ ]:
#功能需求一：
#   数据集展示
#       了解到需要展示的数据集格式大致分为两种，一种是【骨架】，一种是【轨迹】
#       软件可以展示三维环境下的数据，但格式较为严格
#       请各位将展示模型所要用到的数据集的【处理函数】提取出来，稍作修改，将原始数据集
#      处理成软件这边能够接收并处理的格式
#       考虑到不同数据集可能需要不同的呈现效果，软件这边提供可选参数


In [ ]:

#       处理后数据格式：
#           datas，类型为字典
#########################################################################
#               ----必要参数：datas['points']  ！！要绘制的数据点！！
#                   类型为numpy.NDArray, 
#                   shape=[frame, num, XYZcoordinate], 
#                   帧数，点数，坐标。  
#                   表示每帧要绘制的数据点。

#                   例：shape=[106, 25, 3]  即25个点在106帧中，xyz的坐标值
#########################################################################


In [ ]:
#########################################################################
#               ----可选参数：datas['links']    ！！用于【骨架数据】！！
#					类型为list，应该用于【骨架数据】，表示数据点间的连线关系，
#                   内部数据值应为【点的索引】

#					例：[ (0, 1), (1, 11), (3, 4)...]
#########################################################################


In [ ]:
#########################################################################
#               ----可选参数：datas['lines']    ！！用于【轨迹数据】！！
#					类型为list[numpy.NDArray], 
#                   其中每一个NDArray.shape=[num, XYZcoordinate],
#                   点数，坐标。  
#                   表示轨迹线，应该用于【轨迹数据】，
#                   软件将根据该数据将运动点的轨迹用拖尾的形式绘制

#					例： 从上面的例子中抽取点的轨迹  
#                           line1=datas['points'][ : , 0, : ]  #shape=[106,3]
#########################################################################

In [ ]:
#########################################################################
#               如有其他呈现效果需要请单独说明，后续可添加参数
#               如无异议请将数据集处理方式提取成方法

In [1]:
#骨架数据集处理函数示例
#该数据集单个文件内只包含单个对象
import numpy as np

def dealSkeleton(file):
    '''return datas['points'] shape in [frame,point,RGBcoordinate]
        return datas['links'] like [(1,2),(2,21),...]'''
    def read_xyz(file, max_body=2, num_joint=25):
        seq_info = read_skeleton(file)
        data = np.zeros((3, seq_info['numFrame'], num_joint, max_body))
        for n, f in enumerate(seq_info['frameInfo']):
            for m, b in enumerate(f['bodyInfo']):
                for j, v in enumerate(b['jointInfo']):
                    if m < max_body and j < num_joint:
                        data[:, n, j, m] = [v['x'], v['y'], v['z']]
                    else:
                        pass
        return data

    def read_skeleton(file):
        with open(file, 'r', encoding = 'utf-8', errors="ignore") as f:
            skeleton_sequence = {}
            skeleton_sequence['numFrame'] = int(f.readline())
            skeleton_sequence['frameInfo'] = []
            for t in range(skeleton_sequence['numFrame']):
                frame_info = {}
                frame_info['numBody'] = int(f.readline())
                frame_info['bodyInfo'] = []
                for m in range(frame_info['numBody']):
                    body_info = {}
                    body_info_key = [
                        'bodyID', 'clipedEdges', 'handLeftConfidence',
                        'handLeftState', 'handRightConfidence', 'handRightState',
                        'isResticted', 'leanX', 'leanY', 'trackingState'
                    ]
                    body_info = {
                        k: float(v)
                        for k, v in zip(body_info_key, f.readline().split())
                    }
                    body_info['numJoint'] = int(f.readline())
                    body_info['jointInfo'] = []
                    for v in range(body_info['numJoint']):
                        joint_info_key = [
                            'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                            'orientationW', 'orientationX', 'orientationY',
                            'orientationZ', 'trackingState'
                        ]
                        joint_info = {
                            k: float(v)
                            for k, v in zip(joint_info_key, f.readline().split())
                        }
                        body_info['jointInfo'].append(joint_info)
                    frame_info['bodyInfo'].append(body_info)
                skeleton_sequence['frameInfo'].append(frame_info)
        return skeleton_sequence
    
    def get_links():
        
        neighbor_1base = [(1, 2), (2, 21), (3, 21), (4, 3), (5, 21),
                              (6, 5), (7, 6), (8, 7), (9, 21), (10, 9),
                              (11, 10), (12, 11), (13, 1), (14, 13), (15, 14),
                              (16, 15), (17, 1), (18, 17), (19, 18), (20, 19),
                              (22, 23), (23, 8), (24, 25), (25, 12)]
        neighbor_link = [(i - 1, j - 1) for (i, j) in neighbor_1base]
        return neighbor_link
    datas={}
    points=read_xyz(file)
    points=points[:,:,:,0]
    points=np.transpose(points,[1,2,0])
    points[:,:,[0,1,2]]=points[:,:,[2,0,1]]
    points[:,:,0]-=np.median(points[:,:,0])
    points[:,:,2]+=np.abs(np.min(points[:,:,2]))
    
    datas['points']=points
    datas['links']=get_links()
    return datas

path=f'./datas/S001C001P001R001A001.skeleton'
datas=dealSkeleton(path)

In [3]:
print(datas['points'].shape)#该副骨架中25个点在103帧中的三维坐标
print(datas['links'])#25个点的连接关系，两两为一组

(103, 25, 3)
[(0, 1), (1, 20), (2, 20), (3, 2), (4, 20), (5, 4), (6, 5), (7, 6), (8, 20), (9, 8), (10, 9), (11, 10), (12, 0), (13, 12), (14, 13), (15, 14), (16, 0), (17, 16), (18, 17), (19, 18), (21, 22), (22, 7), (23, 24), (24, 11)]


In [5]:
#轨迹数据集处理函数示例：
#该数据集内包含多组对象
def plane_two(data):
    real = {}
    data = data.transpose(1,2,0,3)
    data = data.squeeze()
    for i in range(3):
        temp = np.max(data[:,:,i]) - np.min(data[:,:,i])
        data[:,:,i]= (data[:,:,i] - np.min(data[:,:,i]))/temp*5
    # print(data.shape)
    real['points'] = data
    # print(real['points'].shape)
    line1=data[:,0,:]
    line2=data[:,1,:]
    lines=[line1,line2]
    real['lines']=lines

    return real

data = (np.load("./datas/rechange_val_data_plane_Radar.npy")[10])[0:3]
datas=plane_two(data)

In [7]:
print(datas['points'].shape)#两个点在601帧的三维坐标
for each in datas['lines']:
    print(each.shape)#分别为两个点在601帧的三维坐标，两条轨迹线
#总之就是提供一个函数，能够将原始数据集按照你们的模型处理方法先提取出三维坐标，然后需要轨迹的往里面添加所有轨迹线的各个坐标，需要连接骨架的添加索引

(601, 2, 3)
(601, 3)
(601, 3)


In [9]:
#功能需求二：
#   模型运行
#   软件设计的运行模型的方法为利用子进程的方式，通过命令行运行模型
#   所以需要各位将模型的【运行方式调整成命令行的形式】
#   同时，因为需要在软件上选择数据集后运行，要求模型可以【通过命令行提供数据集为参数】
#   这边有个通过命令行运行模型的示例，请在此代码下【测试】各位的【模型】能否【正常运行并输出】
#   运行结束后（或者出现报错）才会有结果被打印出来，请耐心等待，如果运行时间超过预期，可能是中间出现死循环

#       需要提供的参数为command，directory
#       其中，command为命令行执行的指令
#       directory为模型的运行目录，也可以简单理解为模型代码所在的目录
#       比如：
#       command = f'python /data/home/temp/Desktop/model/st-gcn-cam-org/main.py shapleycam --skeleton S003C001P007R002A019'#记得要像这样能够提供数据集作为参数
#       directory = f'/data/home/temp/Desktop/model/st-gcn-cam-org'
#       这里的directory即为要运行的main.py所在的目录
#
#       测试代码如下：

In [20]:
import subprocess

def CreateTask(command, directory):
    process=subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, cwd=directory)
    return process
def getRunstatu(process):
    exit_code = process.poll()

    if exit_code is None:
        return "子进程仍在运行"
    else:
        return f"子进程已结束，退出码为：{exit_code}"
    
def getReturn(process):
    global stdout, stderr
    Runstatu=getRunstatu(process)
    if Runstatu!="子进程仍在运行":
        stdout, stderr = process.communicate()
        # 检查子进程是否成功完成
        if process.returncode == 0:
            print("Command executed successfully")
            print(stdout)
        else:
            print("Command failed with error")
            print(stderr)
        return 1
    else:
        return 0

#！！！！！！！！！！
stdout, stderr=None,None

if __name__ == '__main__':
    #需要提供的参数command，directory
    # command = f'python /data/home/temp/Desktop/model/st-gcn-cam-org/main.py shapleycam --skeleton S003C001P007R002A019'
    command = f'python /data/home/temp/Desktop/model/st-gcn-cam-org/main.py shapleycam'#原始运行代码？
    command=command.split(' ')
    command.extend('--skeleton S003C001P007R002A019'.split(' '))#追加提供数据集参数
    directory = f'/data/home/temp/Desktop/model/st-gcn-cam-org'
    process=CreateTask(command,directory)
    while not getReturn(process):
        continue

    

Command executed successfully
shapleycam
Using xsub weights.
Processing skeleton: S003C001P007R002A019
各个部位的夏普利值： [[('left_hand', 0.19403504945882277)], [('right_hand', 0.22231027729592182)], [('left_leg', 0.3444637328346931)], [('right_leg', 0.12986051474784824)], [('trunk', 0.1093303529557566)]]
Insertion Curve:
 [0.01042567 0.7766312  0.9999993  0.9999999  0.99992704 0.9999994
 0.9999999  0.9999962  0.99999785 1.         1.         0.9999999
 0.9999999  0.99999976 0.9999999  1.         1.         1.
 0.9999995  0.9999994  0.9999881  0.9999815  0.9999918  0.99998355
 0.9999994  0.99997723 0.9999999  0.99999964 0.9999908  0.9998946
 0.9999908  0.99999964 0.9999999  1.         0.9999999  1.
 1.         1.         1.         1.         1.         0.9999993
 0.9999999  0.9999999  0.9999999  0.9999999  0.9999999  0.9999999
 0.9999999  0.9999999 ]

Insertion AUC: 0.9757354736328125

Deletion Curve:
 [9.9999988e-01 9.9944144e-01 9.9947602e-01 8.7712735e-01 1.5768479e-04
 4.7682218e-07 1.234

In [15]:
#功能需求三：
#   结果展示
#       上面的测试代码中stdout, stderr分别表示标准输出和标准报错的内容
print('-'*20)
print(f'stdout:{stdout}')
print('-'*20)
print(f'stderr:{stderr}')
print('-'*20)

--------------------
stdout:shapleycam
Using xsub weights.
Processing skeleton: S003C001P007R002A019
各个部位的夏普利值： [[('left_hand', 0.19403504945882277)], [('right_hand', 0.22231027729592182)], [('left_leg', 0.3444637328346931)], [('right_leg', 0.12986051474784824)], [('trunk', 0.1093303529557566)]]
Insertion Curve:
 [0.01042567 0.7766312  0.9999993  0.9999999  0.99992704 0.9999994
 0.9999999  0.9999962  0.99999785 1.         1.         0.9999999
 0.9999999  0.99999976 0.9999999  1.         1.         1.
 0.9999995  0.9999994  0.9999881  0.9999815  0.9999918  0.99998355
 0.9999994  0.99997723 0.9999999  0.99999964 0.9999908  0.9998946
 0.9999908  0.99999964 0.9999999  1.         0.9999999  1.
 1.         1.         1.         1.         1.         0.9999993
 0.9999999  0.9999999  0.9999999  0.9999999  0.9999999  0.9999999
 0.9999999  0.9999999 ]

Insertion AUC: 0.9757354736328125

Deletion Curve:
 [9.9999988e-01 9.9944144e-01 9.9947602e-01 8.7712735e-01 1.5768479e-04
 4.7682218e-07 1.23446

In [16]:
#       请各位从中抽取出你们需要在软件的结果界面展示的数据，以字典的形式返回
#       比如：
#           result['AUC']=0.9757
#           result['代码运行时间']=30.197386741638184 秒'
#           ...
#       同时，若模型代码在运行结束后有文件输出，请从代码中提取出每次运行的结果文件路径
#       可以保存在刚才的字典里：
#           result['file_path']=['******.png','******.png',...]
#           最好是绝对路径
#
#       如若有自定义的其他格式，请单独说明

In [17]:
#有任何疑问请及时提出